# Learning Goals

## Optimizing Foundation Models with Supervised Fine-Tuning (SFT)

Often, we want to adapt or customize foundation models to be more performant on our specific task. Fine-tuning refers to how we can modify the weights of a pre-trained foundation model with additional custom data. Supervised Fine-Tuning (SFT) refers to unfreezing all the weights and layers in our model and training on a newly labeled set of examples. We can fine-tune to incorporate new, domain-specific knowledge, or teach the foundation model what type of response to provide. One specific type of SFT is also referred to as “instruction tuning” where we use SFT to teach a model to follow instructions better. In this tutorial, will demonstrate how to perform SFT with Llama3-8b using NeMo 2.0.

NeMo 2.0 introduces Python-based configurations, PyTorch Lightning’s modular abstractions, and NeMo-Run for scaling experiments across multiple GPUs. In this notebook, we will use NeMo-Run to streamline the configuration and execution of our experiments.

## Data
Databricks-dolly-15k is an open-source dataset created by the collaborative efforts of Databricks employees. It consists of high-quality, human-generated prompt/response pairs specifically designed for instruction tuning LLMs. These pairs cover a diverse range of behaviors, from brainstorming and content generation to information extraction and summarization. 

For more information, refer to [databricks-dolly-15k | Hugging Face](https://huggingface.co/datasets/databricks/databricks-dolly-15k).


## Step 1. Import the Hugging Face Checkpoint
We use the `llm.import_ckpt` API to download the specified model using the "hf://<huggingface_model_id>" URL format. It will then convert the model into NeMo 2.0 format. For all model supported in NeMo 2.0, refer to [Large Language Models](https://docs.nvidia.com/nemo-framework/user-guide/24.09/llms/index.html#large-language-models) section of NeMo Framework User Guide.

In [4]:
!uv pip list

Using Python 3.10.16 environment at: /Users/erlebach/src/2025/gordon_nemo/.venv
Package                   Version        Editable project location
------------------------- -------------- ------------------------------------
absl-py                   2.2.2
aiohappyeyeballs          2.6.1
aiohttp                   3.11.18
aiosignal                 1.3.2
aniso8601                 10.0.1
annotated-types           0.7.0
antlr4-python3-runtime    4.9.3
anyio                     4.9.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asciitree                 0.3.3
asttokens                 3.0.0
async-lru                 2.0.5
async-timeout             5.0.1
attrs                     25.3.0
audioread                 3.0.1
babel                     2.17.0
bcrypt                    4.3.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blinker                   1.9.0
boto3                     1.38.

In [5]:
import megatron.core.optimizer

In [1]:
import nemo_run as run
from nemo import lightning as nl
from nemo.collections import llm
from megatron.core.optimizer import OptimizerConfig
import torch
import lightning.pytorch as pl
from pathlib import Path
from nemo.collections.llm.recipes.precision.mixed_precision import bf16_mixed


# llm.import_ckpt is the nemo2 API for converting Hugging Face checkpoint to NeMo format
# example python usage:
# llm.import_ckpt(model=llm.llama3_8b.model(), source="hf://meta-llama/Meta-Llama-3-8B")
#
# We use run.Partial to configure this function
def configure_checkpoint_conversion():
    return run.Partial(
        llm.import_ckpt,
        model=llm.llama3_8b.model(),
        source="hf://meta-llama/Meta-Llama-3-8B",
        overwrite=False,
    )

# configure your function
import_ckpt = configure_checkpoint_conversion()
# define your executor
local_executor = run.LocalExecutor()

# run your experiment
run.run(import_ckpt, executor=local_executor)


/Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TRANSFORMERS_CACHE='/Users/erlebach/.cache/huggingface/hub'
PYTORCH_TRANSFORMERS_CACHE='/Users/erlebach/.cache/huggingface/hub'
HF_MODULES_CACHE='/Users/erlebach/.cache/huggingface/modules'


W0529 21:29:35.624000 87975 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
[NeMo W 2025-05-29 21:29:36 nemo_logging:361] /Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/megatron/core/optimizer/__init__.py:18: UserWarning: Transformer Engine and Apex are not installed. Falling back to Torch optimizers.
      warnings.warn(
    
[NeMo W 2025-05-29 21:29:36 nemo_logging:361] /Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/megatron/core/optimizer/optimizer.py:28: UserWarning: Transformer Engine and Apex are not installed. Falling back to local implementations of multi_tensor_applier and multi_tensor_scale
      warnings.warn(
    
[NeMo W 2025-05-29 21:29:36 nemo_logging:361] /Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/megatron/core/optimizer/clip_grads.py:29: UserWarning: Transformer Engine and Apex are not installed. Falling back to loca

─ Entering Experiment nemo.collections.llm.api.import_ckpt with id: nemo.collections.llm.api.import_ckpt_1748568… ─

Log directory is: /Users/erlebach/.nemo_run/experiments/nemo.collections.llm.api.import_ckpt/nemo.collections.llm.api.import_ckpt_1748568583/nemo.collections.llm.api.import_ckpt


...

Waiting for job nemo.collections.llm.api.import_ckpt-m661959kz100g to finish [log=True]...


mport_ckpt/0 TRANSFORMERS_CACHE='/Users/erlebach/.cache/huggingface/hub'
mport_ckpt/0 PYTORCH_TRANSFORMERS_CACHE='/Users/erlebach/.cache/huggingface/hub'
mport_ckpt/0 HF_MODULES_CACHE='/Users/erlebach/.cache/huggingface/modules'
mport_ckpt/0 W0529 21:29:47.460000 88269 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
mport_ckpt/0 [NeMo W 2025-05-29 21:29:48 nemo_logging:361] /Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/megatron/core/optimizer/__init__.py:18: UserWarning: Transformer Engine and Apex are not installed. Falling back to Torch optimizers.
mport_ckpt/0       warnings.warn(
mport_ckpt/0     
mport_ckpt/0 [NeMo W 2025-05-29 21:29:48 nemo_logging:361] /Users/erlebach/src/2025/gordon_nemo/.venv/lib/python3.10/site-packages/megatron/core/optimizer/optimizer.py:28: UserWarning: Transformer Engine and Apex are not installed. Falling back to local implementations of multi_tensor_app

Job nemo.collections.llm.api.import_ckpt-m661959kz100g finished: FAILED


                                                                                                                   
# The experiment was run with the following tasks: ['nemo.collections.llm.api.import_ckpt']                        
# You can inspect and reconstruct this experiment at a later point in time using:                                  
experiment = run.Experiment.from_id("nemo.collections.llm.api.import_ckpt_1748568583")                             
experiment.status() # Gets the overall status                                                                      
experiment.logs("nemo.collections.llm.api.import_ckpt") # Gets the log for the provided task                       
experiment.cancel("nemo.collections.llm.api.import_ckpt") # Cancels the provided task if still running             
                                                                                                                   

                                                                                                                   
# You can inspect this experiment at a later point in time using the CLI as well:                                  
nemo experiment status nemo.collections.llm.api.import_ckpt_1748568583                                             
nemo experiment logs nemo.collections.llm.api.import_ckpt_1748568583 0                                             
nemo experiment cancel nemo.collections.llm.api.import_ckpt_1748568583 0                                           
                                                                                                                   

## Step 2. Prepare the Data and Customize the DataModule

We will be using Databricks-dolly-15k for this notebook. NeMo 2.0 already provides a `DollyDataModule`. For all data modules that are included in NeMo 2.0, refer to the [data module directory](https://github.com/NVIDIA/NeMo/tree/main/nemo/collections/llm/gpt/data). Example usage:

In [2]:
def dolly() -> run.Config[pl.LightningDataModule]:
    return run.Config(llm.DollyDataModule, seq_length=2048, micro_batch_size=1, global_batch_size=8, num_workers=0)

To use your own data, you will need to create a custom `DataModule`. This involves extending the base class `FineTuningDataModule` so that you have access to existing data handling logic, such as packed sequences. Here we walk you through the process step by step, using the already existing [`DollyDataModule`](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/gpt/data/dolly.py) as an example. 

### Subclass the FineTuningDataModule
You need to extend the `FineTuningDataModule` if you're fine-tuning NeMo models. This provides access to existing data handling logic, such as packed sequences. The `data_root` parameter is where you store your generated `train/validation/test.jsonl` in NeMo format. Below is how `DollyDataModule` does it:

In [ ]:
from typing import List, Optional
from nemo.lightning.io.mixin import IOMixin
from nemo.collections.llm.gpt.data.fine_tuning import FineTuningDataModule

class DollyDataModule(FineTuningDataModule, IOMixin):
    def __init__(
        self,
        seq_length: int = 2048,
        tokenizer: Optional["TokenizerSpec"] = None,
        micro_batch_size: int = 4,
        global_batch_size: int = 8,
        rampup_batch_size: Optional[List[int]] = None,
        force_redownload: bool = False,
        delete_raw: bool = True,
        seed: int = 1234,
        memmap_workers: int = 1,
        num_workers: int = 8,
        pin_memory: bool = True,
        persistent_workers: bool = False,
        pad_to_max_length: bool = False,
        packed_sequence_size: int = -1,
    ):
        self.force_redownload = force_redownload
        self.delete_raw = delete_raw

        super().__init__(
            dataset_root=get_dataset_root("dolly"),
            seq_length=seq_length,
            tokenizer=tokenizer,
            micro_batch_size=micro_batch_size,
            global_batch_size=global_batch_size,
            rampup_batch_size=rampup_batch_size,
            seed=seed,
            memmap_workers=memmap_workers,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=persistent_workers,
            pad_to_max_length=pad_to_max_length,
            packed_sequence_size=packed_sequence_size,
        )

### Override the `prepare_data` Method

The `prepare_data` method is responsible for downloading and preprocessing data if needed. If the dataset is already downloaded, you can skip this step.



In [4]:
def prepare_data(self) -> None:
    # if train file is specified, no need to do anything
    if not self.train_path.exists() or self.force_redownload:
        dset = self._download_data()
        self._preprocess_and_split_data(dset)
    super().prepare_data()

### Implement Data Download and Preprocessing Logic

If your dataset requires downloading or preprocessing, implement this logic within the helper methods. Skip the download part if it's not needed.

In [5]:
def _download_data(self):
    logging.info(f"Downloading {self.__class__.__name__}...")
    return load_dataset(
        "databricks/databricks-dolly-15k",
        cache_dir=str(self.dataset_root),
        download_mode="force_redownload" if self.force_redownload else None,
    )

def _preprocess_and_split_data(self, dset, train_ratio: float = 0.80, val_ratio: float = 0.15):
    logging.info(f"Preprocessing {self.__class__.__name__} to jsonl format and splitting...")

    test_ratio = 1 - train_ratio - val_ratio
    save_splits = {}
    dataset = dset.get('train')
    split_dataset = dataset.train_test_split(test_size=val_ratio + test_ratio, seed=self.seed)
    split_dataset2 = split_dataset['test'].train_test_split(
        test_size=test_ratio / (val_ratio + test_ratio), seed=self.seed
    )
    save_splits['training'] = split_dataset['train']
    save_splits['validation'] = split_dataset2['train']
    save_splits['test'] = split_dataset2['test']

    for split_name, dataset in save_splits.items():
        output_file = self.dataset_root / f"{split_name}.jsonl"
        with output_file.open("w", encoding="utf-8") as f:
            for example in dataset:
                context = example["context"].strip()
                if context != "":
                    # Randomize context and instruction order.
                    context_first = np.random.randint(0, 2) == 0
                    if context_first:
                        instruction = example["instruction"].strip()
                        assert instruction != ""
                        _input = f"{context}\n\n{instruction}"
                        _output = example["response"]
                    else:
                        instruction = example["instruction"].strip()
                        assert instruction != ""
                        _input = f"{instruction}\n\n{context}"
                        _output = example["response"]
                else:
                    _input = example["instruction"]
                    _output = example["response"]

                f.write(json.dumps({"input": _input, "output": _output, "category": example["category"]}) + "\n")

        logging.info(f"{split_name} split saved to {output_file}")

    if self.delete_raw:
        for p in self.dataset_root.iterdir():
            if p.is_dir():
                shutil.rmtree(p)
            elif '.jsonl' not in str(p.name):
                p.unlink()

The original example in Dolly dataset looks like:
```
{'instruction': 'Extract all the movies from this passage and the year they were released out. Write each movie as a separate sentence', 'context': "The genre has existed since the early years of silent cinema, when Georges Melies' A Trip to the Moon (1902) employed trick photography effects. The next major example (first in feature length in the genre) was the film Metropolis (1927). From the 1930s to the 1950s, the genre consisted mainly of low-budget B movies. After Stanley Kubrick's landmark 2001: A Space Odyssey (1968), the science fiction film genre was taken more seriously. In the late 1970s, big-budget science fiction films filled with special effects became popular with audiences after the success of Star Wars (1977) and paved the way for the blockbuster hits of subsequent decades.", 'response': 'A Trip to the Moon was released in 1902. Metropolis came out in 1927. 2001: A Space Odyssey was released in 1968. Star Wars came out in 1977.', 'category': 'information_extraction'}
```
After the preprocessing logic, the data examples are transformed into NeMo format, as below:
```
{'input': "Extract all the movies from this passage and the year they were released out. Write each movie as a separate sentence\n\nThe genre has existed since the early years of silent cinema, when Georges Melies' A Trip to the Moon (1902) employed trick photography effects. The next major example (first in feature length in the genre) was the film Metropolis (1927). From the 1930s to the 1950s, the genre consisted mainly of low-budget B movies. After Stanley Kubrick's landmark 2001: A Space Odyssey (1968), the science fiction film genre was taken more seriously. In the late 1970s, big-budget science fiction films filled with special effects became popular with audiences after the success of Star Wars (1977) and paved the way for the blockbuster hits of subsequent decades.", 'output': 'A Trip to the Moon was released in 1902. Metropolis came out in 1927. 2001: A Space Odyssey was released in 1968. Star Wars came out in 1977.', 'category': 'information_extraction'}
```
Each data example is saved as a json string as one line in the `train/validation/test.jsonl` file, under `data_root` directory you specified earlier.

## Step 3.1: Configure SFT with the NeMo 2.0 API 

In this notebook we use NeMo 2.0 API to perform SFT. First we configure the following components for training. These components are similar between SFT and PEFT. SFT and PEFT both uses `llm.finetune` API. To switch from PEFT to SFT, you just need to remove the `peft` parameter.

### Configure the Trainer
The NeMo 2.0 Trainer works similarly to the PyTorch Lightning trainer.

In [6]:
def trainer() -> run.Config[nl.Trainer]:
    strategy = run.Config(
        nl.MegatronStrategy,
        tensor_model_parallel_size=2
    )
    trainer = run.Config(
        nl.Trainer,
        devices=2,
        max_steps=20,
        accelerator="gpu",
        strategy=strategy,
        plugins=bf16_mixed(),
        log_every_n_steps=1,
        limit_val_batches=2,
        val_check_interval=2,
        num_sanity_val_steps=0,
    )
    return trainer


### Configure the Logger
Configure your training steps, output directories and logging through `NeMoLogger`. In the following example, the experiment output will be saved at `./results/nemo2_sft`.

In [7]:
def logger() -> run.Config[nl.NeMoLogger]:
    ckpt = run.Config(
        nl.ModelCheckpoint,
        save_last=True,
        every_n_train_steps=10,
        monitor="reduced_train_loss",
        save_top_k=1,
        save_on_train_epoch_end=True,
        save_optim_on_train_end=True,
    )

    return run.Config(
        nl.NeMoLogger,
        name="nemo2_sft",
        log_dir="./results",
        use_datetime_version=False,
        ckpt=ckpt,
        wandb=None
    )



### Configure the Optimizer
In the following example, we will be using the distributed adam optimizer and pass in the optimizer configuration through `OptimizerConfig`: 

In [8]:
def adam_with_cosine_annealing() -> run.Config[nl.OptimizerModule]:
    opt_cfg = run.Config(
        OptimizerConfig,
        optimizer="adam",
        lr=5e-6,
        adam_beta2=0.98,
        use_distributed_optimizer=True,
        clip_grad=1.0,
        bf16=True,
    )
    return run.Config(
        nl.MegatronOptimizerModule,
        config=opt_cfg
    )


### Configure the Base Model
We will perform SFT on top of Llama3-8B, so we create a `LlamaModel` to pass to the finetune API.

In [9]:
def llama3_8b() -> run.Config[pl.LightningModule]:
    return run.Config(llm.LlamaModel, config=run.Config(llm.Llama3Config8B))

### Auto Resume
In NeMo 2.0, we can directly pass in the Llama3-8b Hugging Face ID to start SFT without manually converting it into the NeMo checkpoint format, as required in NeMo 1.0.

In [10]:
def resume() -> run.Config[nl.AutoResume]:
    return run.Config(
        nl.AutoResume,
        restore_config=run.Config(nl.RestoreConfig,
            path="nemo://meta-llama/Meta-Llama-3-8B"
        ),
        resume_if_exists=True,
    )


### Configure NeMo 2.0 finetune API
Using all the components we created above, we can call the NeMo 2.0 finetune API. The python example usage is as below:
```
llm.finetune(
    model=llama3_8b(),
    data=dolly(),
    trainer=trainer(),
    log=logger(),
    optim=adam_with_cosine_annealing(),
    resume=resume(),
)
```
We configure the `llm.finetune` API as below:

In [ ]:
def configure_finetuning_recipe():
    return run.Partial(
        llm.finetune,
        model=llama3_8b(),
        trainer=trainer(),
        data=dolly(),
        log=logger(),
        optim=adam_with_cosine_annealing(),
        resume=resume(),
    )

## Step 3.2: Run SFT with NeMo 2.0 API and NeMo-Run

We use `LocalExecutor` for executing our configured finetune function. For more details on the NeMo-Run executor, refer to [Execute NeMo Run](https://github.com/NVIDIA/NeMo-Run/blob/main/docs/source/guides/execution.md) of NeMo-Run Guides. 

In [ ]:
def local_executor_torchrun(nodes: int = 1, devices: int = 2) -> run.LocalExecutor:
    # Env vars for jobs are configured here
    env_vars = {
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
    }

    executor = run.LocalExecutor(ntasks_per_node=devices, launcher="torchrun", env_vars=env_vars)

    return executor

if __name__ == '__main__':
    run.run(configure_finetuning_recipe(), executor=local_executor_torchrun())

## Step 4. Generate Results from Trained SFT Checkpoints

We use the `llm.generate` API in NeMo 2.0 to generate results from the trained SFT checkpoint. Find your last saved checkpoint from your experiment dir: `results/nemo2_sft/checkpoints`. 

In [ ]:
sft_ckpt_path=str(next((d for d in Path("./results/nemo2_sft/checkpoints/").iterdir() if d.is_dir() and d.name.endswith("-last")), None))
print("We will load SFT checkpoint from:", sft_ckpt_path)

When using `llm.generate` API, you can pass a data module such as dolly: `input_dataset=dolly()`. This will use the test set from the specified data module to generate predictions. In the following example, the generated predictions are saved to the `sft_predictions.txt` file. Note that while fine-tuning required a minimum of 2 GPUs with `tensor_model_parallel_size=2`, generating predictions only requires `tensor_model_parallel_size=1`. However, using multiple GPUs can speed up the inference.

In [ ]:
from megatron.core.inference.common_inference_params import CommonInferenceParams


def trainer() -> run.Config[nl.Trainer]:
    strategy = run.Config(
        nl.MegatronStrategy,
        tensor_model_parallel_size=1,
    )
    trainer = run.Config(
        nl.Trainer,
        accelerator="gpu",
        devices=1,
        num_nodes=1,
        strategy=strategy,
        plugins=bf16_mixed(),
    )
    return trainer

def configure_inference():
    return run.Partial(
        llm.generate,
        path=str(sft_ckpt_path),
        trainer=trainer(),
        input_dataset=dolly(),
        inference_params=CommonInferenceParams(num_tokens_to_generate=20, top_k=1),
        output_path="sft_prediction.jsonl",
    )


def local_executor_torchrun(nodes: int = 1, devices: int = 1) -> run.LocalExecutor:
    # Env vars for jobs are configured here
    env_vars = {
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
    }

    executor = run.LocalExecutor(ntasks_per_node=devices, launcher="torchrun", env_vars=env_vars)

    return executor

if __name__ == '__main__':
    run.run(configure_inference(), executor=local_executor_torchrun())


After the inference is complete, you will see results similar to the following:

In [ ]:
%%bash
head -n 3 sft_prediction.jsonl

You should see output similar to the following:
```
{"input": "What is best creator's platform", "category": "brainstorming", "label": "Youtube. Youtube should be best creator platform", "prediction": " for video content creators. YouTube is best creator's platform for video content creators."}
{"input": "When was the last time the Raiders won the Super Bowl?", "category": "open_qa", "label": "The Raiders have won three Super Bowl championships (1977, 1981, and 1984), one American Football League (AFL) championship (1967), and four American Football Conference (AFC) titles. The most recent Super Bowl ring was won in 1984 against the Washington Redskins of the NFC.", "prediction": " 2003"}
{"input": "Muckle Water is a long, narrow fresh water loch on Ward Hill on Rousay, Orkney, Scotland. It is the biggest loch on the island and is popular for fishing. It can be reached by a track from the roadside. The Suso Burn on the north eastern shore drains the loch into the Sound of Rousay.\n\nWhere is Muckle Water?", "category": "closed_qa", "label": "Muckle water is located in Rousay, Orkney, Scotland.", "prediction": " Muckle Water is a long, narrow fresh water loch on Ward Hill on Rousay,"}
```

## Step 5. Calculate Evaluation Metrics

We can evaluate the model's predictions by calculating the Exact Match (EM) and F1 scores.
- Exact Match is a binary measure (0 or 1) checking if the model outputs match one of the
ground truth answer exactly.
- F1 score is the harmonic mean of precision and recall for the answer words.

Below is a script that computes these metrics. The sample scores can be improved by training the model further and performing hyperparameter tuning. In this notebook, we only train for 20 steps.

In [ ]:
!python /opt/NeMo/scripts/metric_calculation/peft_metric_calc.py --pred_file sft_prediction.jsonl --label_field "label" --pred_field "prediction"